In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("spam.csv")
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
data.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [4]:
data['spam']=data['Category'].apply(lambda x: 1 if x=='spam' else 0 )

In [5]:
data.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data.Message,data.spam)

### Apply the feature extraction in the training values as machine can't understand without 0 or 1

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vec=CountVectorizer()

In [9]:
x_train_count=vec.fit_transform(x_train.values)
x_train_count.toarray()[:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### As the feature is  like binary movement(0 or 1) we will first use the Bernoulli distribution 

In [10]:
from sklearn.naive_bayes import BernoulliNB
model=BernoulliNB()

In [11]:
model.fit(x_train_count,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [12]:
# also apply fit tranform for the x_test
model.score(vec.transform(x_test),y_test)

0.9813352476669059

### Making the MultinomialNB

In [13]:
from sklearn.naive_bayes import MultinomialNB
model_nb=MultinomialNB()
model_nb.fit(x_train_count,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
model_nb.score(vec.transform(x_test),y_test)

0.9913854989231874

### Cross Validation


In [20]:
from sklearn.model_selection import cross_val_score
import numpy as np
cross_val_score(MultinomialNB(),x_train_count,y_train,cv=9)

array([0.96989247, 0.98064516, 0.97849462, 0.98060345, 0.98491379,
       0.9612069 , 0.98275862, 0.98275862, 0.9762931 ])

In [21]:
np.average(cross_val_score(MultinomialNB(),x_train_count,y_train,cv=9))

0.9775074156470152

### Sklearn Pipeline

In [22]:
from sklearn.pipeline import Pipeline

In [23]:
pipe=Pipeline([('vectorizer',CountVectorizer()),('nb',MultinomialNB())])

In [25]:
pipe.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('nb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [27]:
pipe.score(x_test,y_test)

0.9913854989231874

#### Cross validation & Pipeline

In [28]:
cross_val_score(pipe,x_train,y_train,cv=6)

array([0.9784792 , 0.98134864, 0.98421808, 0.97557471, 0.98706897,
       0.98563218])